In [19]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('stopwords')
nltk.download('wordnet')


C:\Users\alvas\AppData\Local\Temp\ipykernel_18648\763674205.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alvas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alvas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [21]:
train = pd.read_csv("kg_train.csv")
test = pd.read_csv("kg_test.csv")

train.head()


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [22]:
x = train['text']
y = train['label']

x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size= 0.2, random_state= 42, stratify = y
)

In [23]:
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)


In [24]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


In [25]:
def clean_text(text):
    text = re.sub(r"<.*?>", " ", text)          # eliminar HTML
    text = re.sub(r"[^a-zA-Z]", " ", text)      # solo letras
    text = text.lower()                         # minúsculas
    text = re.sub(r"\s+", " ", text).strip()    # espacios extra
    return text


In [26]:
x_train_clean = x_train.apply(clean_text)
x_val_clean = x_val.apply(clean_text)

In [27]:
stopwords = set(stopwords.words('english'))

def remove_stopwords(text):
    return ''.join(
        word for word in text.split() if word not in stopwords
    )

In [28]:
x_train_clean = x_train_clean.apply(remove_stopwords)
x_val_clean = x_val_clean.apply(remove_stopwords)


In [29]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ''.join(
        lemmatizer.lemmatize(word) for word in text.split()
    )

In [30]:
x_train_clean = x_train_clean.apply(lemmatize_text)
x_val_clean = x_val_clean.apply(lemmatize_text)


In [33]:
bow = CountVectorizer(max_features=2000)
x_bow = bow.fit_transform(x_train_clean)

bow_df = pd.DataFrame(
    x_bow.toarray(),
    columns=bow.get_feature_names_out()
)

bow_df.head()


,abedinhumafridayaugustamreericjohn,abedinhumafridayjuneamhfyi,abedinhumafridaymaypmcallscallcheryldurbincalledubackavailablenexthoursreturnbarakcallmitchelltalkedgetcallreadoutfirstudecideuwanttotalk,abedinhumamondayaugustpmhb,abedinhumamondayjulyamfwlettermeetingafghanministerlaborsocialaffiarsmartyrsanddisabledmolsamdletterjpgmolsamdletterpagedocjpg,abedinhumamondayjulypmoscarlatetonitegetdctillmidnightfyilrfl,abedinhumasaturdayaprilpmredprkfinalpjtextamsurefinepass,abedinhumasaturdayaugustpmhfwreutersgermanysteinmeierproposesafghanpulltalks,abedinhumasaturdaydecemberpmrereuterstalibanplanneduseamericanspakistanattacks,abedinhumasaturdaymarchamreabumazentheyoriginalemailsentpastingoscarprintedb,...,ytcinvitationchinadearsirmadampleasurewritetoyourespectcompanyyokoonotradingcompanysmallcompanylocatedchinaintoinnovativeartisticexampleschineseartworksrangingpreciousmetalssculptureandpaintingspreciousartifactsrangehandynastysongdynastyhighlyauthenticheldfirstexhibitionauctionwesoldlotartifactsgeneralpublicalongperiodcollectingartifactspurposesofviewingexhibitionhighlysuccessfullotforeignbuyersparticipatedinthesalesboughtsculpturepaintingshoweversomeartifactspaidbytheseclientsmuchreleifindicatedinterestpayinggoodsproblemofdistancelanguagebarriersmentiongovernmenttradeoutsidechinaaresearchingrepresentativeshelpusestablishmediumgettingfundsourcostumerscanadaamericaeuropewellasmakingpaymentsuspleaseyouareinterestedtransactingbusinessviewhelpingusclientscouldmakepaymentarepresentativeusgladandadequatecompensationsgivenotherbenefitspleasecontactukbasedexecutiverepresentativecoordinatormrandrewwoodsemailwoodsandrewyahoocomtelfaxsubjectsatisfactiongiventheopportunitynegotiatemodepaymentforyourservicesrepresentativeincanadaamericaeuropeinterestedforwardhimyourphonenumberfaxfullcontactaddressesregardsmryokoonoexecutivedirectormailsentwebmailservicephpnukepoweredsitehttpyoursitecom,yukosoilrussianlargestoilcompanychairmanceomenatepsbpbankwellreputablefinancialinstitutionbranchesworldsourcefundsdiprofilingamountexcessusseekyourpartnershipaccommodatingdyourewardedtotalsumpartnershipoupartnerdintroductionselfpersonalconsultantauthorityhandedtransfermoneyamericanoilmerchant,yup,...,zenithhousehydeedgwareroadlondonnweedisclaimerconfidentiallyemailfilestransmittedconfidentialintendedsolelyuseindividualentityaddressedreceivedemailerrorpleasedeleteimmediatelyunlessintendedrecipientmayusedisclosecopydistributeemailattachmentscontentsemailfilestransmittedmaysubjectcopyrightlawscasecontentmaycopiedadapteddistributedtransmittedwithoutadvanceconsentcopyrightownervirusesdagocrepresentwarrantfilesattachedemailfreecomputervirusesdefectsfilesprovidedmayusedbasisuserassumesresponsibilitylossdamagearisingusefiles,zenithhousehydeedgwareroadlondonnweegreetings,...,zhangbankchinashenzhenchinadearfriendzhangbankmanagerbankchinashenzhenbranchchinaurgentconfidentialbusinesspropositionjuneconsultantcontractorchinesesolidmineralscorporationmadenumberedtimefixeddeposittwelvecalendarmonthsvaluedtwentyonemilliondollarsusbranchuponmaturitysentroutinenotificationforwardingaddressgotreplymonthsentreminderfinallydiscoveredcontractemployerssouthkoreasolidmineralscorporationdiedalongwifeplanecrashinvestigationfounddiedwithoutmakingattemptstracenextkinfruitlessthereforemadeinvestigationdiscovereddeclarenextkinrelationsofficialdocumentsincludingbankdepositpaperworkbanksumusstillsittingbankinterestrolledprincipalsumendyearoneevercomeforwardclaimaccordinglawschinaexpirationeightyearsmoneyrevertownershipsouthkoreagovernmentnobodyappliesclaimfundsconsequentlyproposallikeforeignerassistreceivingmoneyeverymachinerysetenablereceivermoneyviafinancefirmbasedoverseasneedurgentassistancemoneygethandscorruptgovernmentofficialssimplerequiresendcompletenamescontactaddresstelephonefaxnumbersableapplynecessarydocumentsenablereceivemoneyviafinancefirmoverseasnamereceivingmoneysharerationoteprojectriskfreeconsideringpositioncreditofficerbankguaranteesuccessfulexecutiontransactionfacilitiesobta

In [34]:
spam_words = bow_df[y_train == 1].sum().sort_values(ascending=False).head(10)
ham_words = bow_df[y_train == 0].sum().sort_values(ascending=False).head(10)

spam_words, ham_words


(pmarginpxpaddingpxbody                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [37]:
tfidf = TfidfVectorizer(max_features=3000)

x_train_tfidf = tfidf.fit_transform(x_train_clean)
x_val_tfidf = tfidf.transform(x_val_clean)


In [44]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train_tfidf, y_train)


LogisticRegression(max_iter=1000)

In [43]:
y_pred = model.predict(x_val_tfidf)

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))


[[677   0]
 [432  84]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       677
           1       1.00      0.16      0.28       516

    accuracy                           0.64      1193
   macro avg       0.81      0.58      0.52      1193
weighted avg       0.78      0.64      0.55      1193



In [45]:
test_clean = test["text"].apply(clean_text)
test_clean = test_clean.apply(remove_stopwords)
test_clean = test_clean.apply(lemmatize_text)

X_test_tfidf = tfidf.transform(test_clean)

test["prediction"] = model.predict(X_test_tfidf)
test.head()


,text,prediction
0,usiness is for the fact that the deceased man ...,0
1,They are happy to adjust to the afternoon. I a...,0
2,Lael Brainard was confirmed 78-19 this afterno...,0
3,H <hrod17@clintonemail.com>Friday March 26 201...,0
4,"n;""> Dear Good Friend,<br><br><br>I am happy t...",0
